In [1]:
import pandas as pd
import geopandas as gpd
from keplergl import KeplerGl
import json

/home/jptalusan/anaconda3/envs/py39/lib/python3.9/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
fp = '/home/jptalusan/gits/mta_simulator_redo/code_root/scenarios/baseline/data/stops_node_matching.pkl'
stops_info = pd.read_pickle(fp)
stops_info.head(1)

,stop_sequence,stop_id_original,stop_name,map_latitude,map_longitude,nearest_node,nearest_edge
0,24,12ADEMNN,12TH AVE S & DEMONBREUN ST NB,36.154509,-86.786056,202342839,"(202284281, 202342839, 0)"


In [3]:
# fp = '../code_root/visualizer.csv'
# viz_df = pd.read_csv(fp)
# viz_df = viz_df.fillna(method='backfill').sort_values(by='time')
# viz_df = pd.merge(viz_df, stops_info[['stop_id_original', 'map_latitude', 'map_longitude']], left_on='last_visited_stop', right_on='stop_id_original')

# map_1 = KeplerGl(height=800)
# map_1.add_data(viz_df, name='buses')

# with open('/home/jptalusan/gits/mta_simulator_redo/code_root/scenarios/baseline/output/config.json', 'r') as fp:
#     config = json.load(fp)
# map_1.config = config
    
# map_1

In [4]:
# config = map_1.config
# with open('/home/jptalusan/gits/mta_simulator_redo/code_root/scenarios/baseline/output/config.json', 'w') as fp:
#     json.dump(config, fp)

In [5]:
fp = '../code_root/visualizer_1.csv'
viz_df = pd.read_csv(fp)
viz_df = viz_df.fillna(method='backfill').sort_values(by='time')
viz_df = pd.merge(viz_df, stops_info[['stop_id_original', 'map_latitude', 'map_longitude']], left_on='last_visited_stop', right_on='stop_id_original')
viz_df = viz_df.drop_duplicates()

In [6]:
map_1 = KeplerGl(height=800)
map_1.add_data(viz_df, name='buses')

# with open('/home/jptalusan/gits/mta_simulator_redo/code_root/scenarios/baseline/output/config.json', 'r') as fp:
#     config = json.load(fp)
# map_1.config = config
    
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'buses': {'index': [0, 1, 2, 3, 4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22,…